In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
import time
from tqdm import tqdm
from itertools import permutations
import pandas as pd

import matplotlib as mpl
import datetime
import math
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

from statsmodels.iolib.summary2 import summary_col

from scipy import stats



In [ ]:
## for initializing the SIR sub-population of each node
def decomposition(i,k):
    while k > 0:
        if k ==1:
            n = i
        else:
            n = random.randint(0, i)
        yield n
        i -= n
        k -= 1

In [ ]:
## initializtion_mobility_network
def initializtion_mobility_network(number_of_node, average_in_degree, average_out_degree):
    ### node generation
    num_node = number_of_node
    G_saved = nx.DiGraph()

    ##generate the node 
    G_saved.add_nodes_from(nx.path_graph(num_node))
    for i in range(len(G_saved.nodes())):
        # S, I, R
        num_pop = round(np.random.normal(10000,2500,1)[0])
        G_saved._node[i].update({'cont_popu': num_pop})
        G_saved._node[i].update({'status': [num_pop,0,0]})
        G_saved._node[i].update({'record':[G_saved._node[i]['status']]})
        G_saved._node[i].update({'stayed': num_pop})
        G_saved._node[i].update({'tem_popu':num_pop})
        G_saved._node[i].update({'tem_status':[0,0,0]})
        G_saved._node[i].update({'tem_popu_rate':[st/num_pop for st in G_saved._node[i]['status']]})

    initial_infected_list = random.sample(range(len(G_saved.nodes())), 5)
    for i in initial_infected_list:
        tem =nx.get_node_attributes(G_saved,'status')[i]
        G_saved._node[i].update({'status':[tem[0]-10, tem[1]+10, 0]})
        G_saved._node[i].update({'record':[G_saved._node[i]['status']]})


    sorted_popu={k: v for k, v in sorted(nx.get_node_attributes(G_saved, 'cont_popu').items(), key=lambda item: item[1],reverse=True)}
    
    print('initially infected towns list: ', initial_infected_list)
    print('information of initially infected towns list:', [nx.get_node_attributes(G_saved,'status')[i] for i in initial_infected_list])

    ###### edge generation

    # for out edges of the mobility network
    G_o = nx.barabasi_albert_graph(num_node,average_out_degree)
    # for in edges of the mobility network
    G_i = nx.barabasi_albert_graph(num_node,average_in_degree)

    sorted_o = [ node for (node, val) in sorted(G_o.degree(), key=lambda pair: pair[1],reverse=True )]
    sorted_o_map =  dict(zip(sorted_o ,sorted_popu.keys()))
    G_o = nx.relabel_nodes(G_o, sorted_o_map )
    sorted_i = [ node for (node, val) in sorted(G_i.degree(), key=lambda pair: pair[1],reverse=True )]
    sorted_i_map =  dict(zip(sorted_i ,sorted_popu.keys()))
    G_i = nx.relabel_nodes(G_i, sorted_i_map )

    ## set the max_weight of each edge based on the population
    max_weight_list = [round(list(nx.get_node_attributes(G_saved,'cont_popu').values())[i]/(num_node/3)) for i in range(num_node)]
    for edge_added in G_i.edges():
        G_saved.add_edge(edge_added[1], edge_added[0], weight=random.sample(range(10,max_weight_list[edge_added[1]]),1)[0])
    for edge_added in G_o.edges():
        G_saved.add_edge(edge_added[0], edge_added[1], weight=random.sample(range(10,max_weight_list[edge_added[0]]),1)[0])

    ### add edges if some node has no in-mobility or out-mobnility to avoid the isolaiton
    for i in G_saved.out_degree():
        if i[1] == 0:
            #print('no out nodes: ',  i[0])
            for k in random.sample([m for m in range(num_node) if m != i[0]],5):
                #print(k)
                G_saved.add_edge(i[0], k,weight=random.sample(range(10,max_weight_list[i[0]]),1)[0]) 
    for i in G_saved.in_degree():
        if i[1] == 0:
            #print('no in nodes: ', i[0])
            for k in random.sample([m for m in range(num_node) if m != i[0]],5):
                #print(k)
                G_saved.add_edge(k, i[0],weight=random.sample(range(10,max_weight_list[k]),1)[0])
    
    return(G_saved)

In [ ]:
def SIR_metapopulation_simulation(G_input,duration_input, edg_changed_input, wei_changed_input):
    for i in range(len(G_input)):
        G_input._node[i].update({'record':[G_input._node[i]['status']]})

    edge_martix_record = []
    num_node = len(G_input)
#     for i in range(num_node):
#         G_input._node[i].update({'record':[G_input._node[i]['status']]})
    G = G_input.copy()
    ## set the max_weight of each edge based on the population
    max_weight_list = [round(list(nx.get_node_attributes(G,'cont_popu').values())[i]/(num_node/3)) for i in range(num_node)]
    
    ## the parameters for the dynamic mobility netwoek
    edg_changed_rate = edg_changed_input
    wei_changed_rate = wei_changed_input
    
    ### simualtion 
    total_edge_change_pro = 1
    edge_added_record = [len(G.edges)]
    duration = duration_input
    for time in range(duration):
        if time ==0:
            edge_pool =  list(permutations(range(len(G.nodes())), 2))
            edge_added = list(G.edges)
        ### the rest process
        else:
            if total_edge_change_pro < 1:
                check_have_in_out = -1
                while check_have_in_out <0:
                    check_have_in_out = 1
                    edge_modified = random.sample(edge_added,round(len(edge_added)-
                                                                   len(edge_added)* total_edge_change_pro))
                    for check_edge in edge_modified:
                        if G.in_degree(check_edge[1]) <3 or G.out_degree(check_edge[0])<3:
                            check_have_in_out = -1
                G.remove_edges_from(edge_modified)
                difference_pool = list(set(edge_added).difference(set(edge_modified)))
                edge_added = difference_pool
            if total_edge_change_pro > 1:
                complement_pool = list(set(edge_pool) - (set(edge_added)))
                edge_modified = random.sample(complement_pool,-round(len(edge_added)
                                                                     -len(edge_added)* total_edge_change_pro))
                for sro, tar in edge_modified:
                    G.add_edge(sro, tar, weight=random.sample(range(1,max_weight_list[sro]),1)[0])
                edge_added += edge_modified
            edge_added_record.append(len(edge_added))

            ## update the weight of the edge
            for i in G.edges():
                tem_w = G.edges()[i]['weight']
                G.edges()[i]['weight'] = max(10, round(tem_w*homo_weight_change_pro))


        ## update the tem information
        ## calculate the number of stayed
        weight_list =nx.get_edge_attributes(G, 'weight')
        for i in range(len(G.nodes())):
            G._node[i].update({'stayed': G._node[i]['cont_popu']- G.out_degree(weight = 'weight')[i]})

        ## calculate the distribution of stayed people
        for i in range(len(G.nodes())):
             G._node[i].update({'tem_status': [G._node[i]['stayed']*sir for sir in G._node[i]['tem_popu_rate']]})

        ## calculate the new pop        
        for edg in  G.edges():
            tem_in_node = edg[1]
            tem_out_node = edg[0]
            ## the distrbution of go out people per link 
            tem_out_dis = [weight_list[edg]* dis for dis in G._node[edg[0]]['tem_popu_rate']]
            pre_dis =  G._node[edg[1]]['tem_status'].copy()
            G._node[edg[1]].update({'tem_status':[a + b for a, b in zip(pre_dis, tem_out_dis)]})

        for i in range(len(G.nodes())):
             G._node[i].update({'tem_popu': sum( G._node[i]['tem_status'])})

        ##SIR model
        in_ra_s = 0.25
        in_ra_m = 3
        re_ra = 0.25
        for node in range(len(G.nodes())):
            stayed_S_to_I = G._node[node]['stayed']* G._node[node]['tem_popu_rate'][0]*G._node[node]['tem_status'][1]/G._node[node]['tem_popu']*in_ra_s
            out_S_to_I = 0
            for out_edg in G.out_edges([node]):
                des = out_edg[1]
                tem_out_S_to_I = weight_list[out_edg]* G._node[node]['tem_popu_rate'][0]*G._node[des]['tem_status'][1]/G._node[des]['tem_popu']*in_ra_m
                out_S_to_I += tem_out_S_to_I
            #print(node,stayed_S_to_I,out_S_to_I)
            S_to_I = round(stayed_S_to_I + out_S_to_I)
            I_to_R = round(G._node[node]['cont_popu']* G._node[node]['tem_popu_rate'][1]*re_ra)
            pre_status = G._node[node]['status'].copy()
            G._node[node].update({'status': [pre_status[0]-S_to_I,pre_status[1]+S_to_I-I_to_R,pre_status[2]+I_to_R]})
            G._node[node]['record'].append(G._node[node]['status'])

        ## record the edges information overtime
        tem_edge_matrix = nx.adjacency_matrix(G).A
        for node in range(len(G.nodes())):
            tem_edge_matrix[node][node] += G._node[node]['stayed']
        edge_martix_record.append(tem_edge_matrix)


        #G.remove_edges_from(list(G.edges()))
        for i in range(len(G.nodes())):
            G._node[i].update({'tem_popu_rate':[st/G._node[node]['cont_popu'] for st in G._node[i]['status']]})

        whole_R_changed = sum([nx.get_node_attributes(G,'record')[i][time+1][1]-nx.get_node_attributes(G,'record')[i][time][1] for i in range(len(G))])
        total_edge_change_pro = round(math.e**(-whole_R_changed/(2000*edg_changed_rate)),2)
        homo_weight_change_pro = round(math.e**(-whole_R_changed/(2000*wei_changed_rate)),2)

    stayed_record = []
    for j in range(duration):
        stayed_record.append(sum([edge_martix_record[j][i][i]/G._node[i]['cont_popu'] for i in range(num_node)])/(num_node))

    I_record = []
    for i in range(duration):
        tem_I = 0
        for j in range(num_node):
            tem_check = nx.get_node_attributes(G,'record')[j]
            tem_I += tem_check[i][1]
        I_record.append(tem_I)

    tem = nx.from_numpy_matrix(np.matrix(edge_martix_record[0]), create_using=nx.DiGraph)
    tem.remove_edges_from(nx.selfloop_edges(tem))
    nx.set_edge_attributes(tem, 0, "anti_weight")
    for i in tem.edges:
        tem.edges[i]['anti_weight'] = 1/ tem.edges[i]['weight']

    # save the data
    data_record =  pd.DataFrame(columns = ['town','day', 'population','S', 'I', 'R', 
                                           'in_flow', 'out_flow', 'intra_flow','in_link', 'out_link',
                                          'In_pr', 'Out_pr','Bet_wei','de_cen', 'in_de_cen', 'out_de_cen',
                                           'all_I','I_t_1'])
    for dd in range(duration):
        ## geneate the network and calculate the network centrality
        tem = nx.from_numpy_matrix(np.matrix(edge_martix_record[dd]), create_using=nx.DiGraph)
        tem.remove_edges_from(nx.selfloop_edges(tem))
        nx.set_edge_attributes(tem, 0, "anti_weight")
        tem_all_I = sum([nx.get_node_attributes(G,'record')[td][dd][1] for td in range(num_node)])
        for i in tem.edges:
            tem.edges[i]['anti_weight'] = 1/ tem.edges[i]['weight']
        tem_in_pr = nx.pagerank(tem,alpha=0.8)
        tem_out_eigen = nx.pagerank(tem.reverse(copy=True), weight = 'weight' )
        tem_bet_weight_cen = nx.betweenness_centrality(tem, weight  = 'anti_weight')
        tem_de_cen  =  nx.degree_centrality(G)
        tem_de_in_cen = nx.in_degree_centrality(G)
        tem_de_out_cen = nx.out_degree_centrality(G)
        for tt in range(num_node):
            tem_used = [tt, dd, G._node[tt]['cont_popu']]
            tem_used += nx.get_node_attributes(G,'record')[tt][dd]

            tem_out_flow =sum(edge_martix_record[dd][tt])-edge_martix_record[dd][tt,tt]
            tem_intra_flow =edge_martix_record[dd][tt,tt]
            tem_in_flow =edge_martix_record[dd][:, tt].sum()-edge_martix_record[dd][tt,tt]
            tem_out_link = np.count_nonzero(edge_martix_record[dd][tt])-1
            tem_in_link = np.count_nonzero(edge_martix_record[dd][:, tt])-1
            tem_used.append(tem_in_flow)
            tem_used.append(tem_out_flow)
            tem_used.append(tem_intra_flow)
            tem_used.append(tem_in_link)
            tem_used.append(tem_out_link)
            tem_used.append(tem_in_pr[tt])
            tem_used.append(tem_out_eigen[tt])
            tem_used.append(tem_bet_weight_cen[tt])
            tem_used.append(tem_de_cen[tt])
            tem_used.append(tem_de_in_cen[tt])
            tem_used.append(tem_de_out_cen[tt])
            tem_used.append(tem_all_I)
            if dd ==0:
                tem_used.append(0)
            else:
                tem_used.append(nx.get_node_attributes(G,'record')[tt][dd-1][1])

            data_record .loc[len(data_record )]=tem_used
            
        
    return(data_record)


# Mobility Network Generation and simulation (with Example)

In [ ]:
## generat the initial mobility network
### input:  number_of_node, average_in_degree, average_out_degree
G_saved = initializtion_mobility_network(50, 10, 10)

In [ ]:
## run the SIR model
### input: initial mobility network, duration, change_rate_of_edge, change_rate_of_weight
tem = SIR_metapopulation_simulation(G_saved, 10, 25, 50)


In [ ]:
tem

# Four scenarios study

In [ ]:
### the paramter setting for four scenarios study
scenario_num_list = [[200,500],[100,250], [50,100], [25,50]]
scenario_fir_record = [[],[],[]] 
scenario_sec_record = [[],[],[]] 
scenario_thi_record = [[],[],[]]
scenario_fou_record = [[],[],[]] 
scenario_whole_record = [scenario_fir_record,scenario_sec_record,scenario_thi_record,scenario_fou_record]

In [ ]:
for round_num in tqdm(range(5)):
    G_tem = initializtion_mobility_network(50, 10, 10)
    for sc_num in range(len(scenario_num_list)) :
        data_record = SIR_metapopulation_simulation(G_tem, 50, scenario_num_list[sc_num][0], scenario_num_list[sc_num][1])
        
        data=data_record.copy()
        basic = ['all_I', 'in_flow']
        basic_in = basic[0]
        for tem_ad in range(len(basic)-1):
            basic_in += " + " 
            basic_in += basic[tem_ad+1]
        basic_mod = ols(formula='I_t_1 ~ {} + C(town)'.format(basic_in), data=data)
        basic_res = basic_mod.fit(cov_type='cluster',cov_kwds={'groups': data['town']},use_t=True)
        tem_basic_line = basic_res.rsquared_adj
        para_name_used = ['In_pr', 'Out_pr','Bet_wei']
        for net_pa_in in range(len(para_name_used)) :
            data=data_record.copy()
            basic =['all_I', 'in_flow'] + [para_name_used[net_pa_in]]
            basic_in = basic[0]
            for tem_ad in range(len(basic)-1):
                basic_in += " + " 
                basic_in += basic[tem_ad+1]   
            developed_mod = ols(formula='I_t_1 ~ {} + C(town)'.format(basic_in), data=data)
            developed_res = developed_mod.fit(cov_type='cluster',cov_kwds={'groups': data['town']},use_t=True)
            developed_res_added  =  developed_res.rsquared_adj
            scenario_whole_record[sc_num][net_pa_in].append(developed_res_added/tem_basic_line)

# Show the result table

In [ ]:
tem_test = scenario_whole_record.copy()
for i in range(4):
    for j in range(3):
        tem_test[i][j] = [k-1 for k in tem_test[i][j]]

data_showed =  pd.DataFrame(columns = ['Edge_changed_rate','Weight_changed_rate','In_pr', 'Out_pr','Bet_wei'], index = ['slow', 'medium', 'fast', 'sharp'])
data_showed['Edge_changed_rate'] = [200,100,50,25]
data_showed['Weight_changed_rate'] = [500,250,100,50]
for i in range(4):
    for j in range(3):
        tem_mean = round((np.mean(tem_test[i][j]))*100,2)
        tem_std = round(np.std(tem_test[i][j])*100,2)
        data_showed.iloc[i,j+2] = '{}% \u00B1 {}%'.format(tem_mean,tem_std)

In [ ]:
data_showed